# Advanced Musical Chord Laboratory

This notebook provides a comprehensive and professional environment for experimenting with musical chords and progressions, featuring advanced sound generation techniques.

In [ ]:
# Import required libraries
import sys
import subprocess

# Check Python environment
print(f"Python version: {sys.version}")
print(f"Python executable: {sys.executable}")

# Install required packages if needed
def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Try importing required libraries, install if not found
try:
    from pydub.generators import Sine, Sawtooth, Square
    from pydub import AudioSegment
except ImportError:
    print("Installing pydub...")
    install_package('pydub')
    from pydub.generators import Sine, Sawtooth, Square
    from pydub import AudioSegment

# Check for SoX (Sound eXchange) for advanced audio processing
try:
    subprocess.run(['sox', '--version'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
except (subprocess.CalledProcessError, FileNotFoundError):
    print("Installing SoX for advanced audio effects...")
    subprocess.run(['sudo', 'dnf', 'install', '-y', 'sox'], check=True)

print("\nEnvironment check complete!")

## Musical Theory Background

### Advanced Chord Types and Their Emotional Impact

- **Basic Triads**:
  - Major: Happy, confident, resolved
  - Minor: Sad, contemplative, longing
  - Diminished: Tense, suspenseful, uneasy
  - Augmented: Unsettling, mysterious

- **Extended Chords**:
  - 7th Chords: Tension seeking resolution
  - 9th Chords: Complex, jazzy, sophisticated

- **Suspended Chords**:
  - Sus2: Open, airy, floating
  - Sus4: Dreamy, anticipating

### Waveform Characteristics
- **Sine Wave**: Pure, clean tone
- **Sawtooth Wave**: Bright, harsh, electronic
- **Square Wave**: Hollow, buzzy, synthesizer-like

In [2]:
# Note frequencies (in Hz) for easy reference
NOTES = {
    'C4': 261.63, 'C#4': 277.18, 'Db4': 277.18, 'D4': 293.66,
    'D#4': 311.13, 'Eb4': 311.13, 'E4': 329.63, 'F4': 349.23,
    'F#4': 369.99, 'Gb4': 369.99, 'G4': 392.00, 'G#4': 415.30,
    'Ab4': 415.30, 'A4': 440.00, 'A#4': 466.16, 'Bb4': 466.16,
    'B4': 493.88, 'C5': 523.25, 'D5': 587.33, 'E5': 659.25
}

# Pre-defined chord structures with extended chords
CHORDS = {
    # Basic Triads
    'C': [NOTES['C4'], NOTES['E4'], NOTES['G4']],  # C major
    'Cm': [NOTES['C4'], NOTES['Eb4'], NOTES['G4']],  # C minor
    'Cdim': [NOTES['C4'], NOTES['Eb4'], NOTES['Gb4']],  # C diminished
    'Caug': [NOTES['C4'], NOTES['E4'], NOTES['G#4']],  # C augmented
    
    # Suspended Chords
    'Csus2': [NOTES['C4'], NOTES['D4'], NOTES['G4']],  # C suspended 2nd
    'Csus4': [NOTES['C4'], NOTES['F4'], NOTES['G4']],  # C suspended 4th
    
    # Extended Chords
    'C7': [NOTES['C4'], NOTES['E4'], NOTES['G4'], NOTES['Bb4']],  # C dominant 7th
    'Cmaj7': [NOTES['C4'], NOTES['E4'], NOTES['G4'], NOTES['B4']],  # C major 7th
    'C9': [NOTES['C4'], NOTES['E4'], NOTES['G4'], NOTES['Bb4'], NOTES['D5']],  # C 9th
    
    # Other Common Chords
    'Am': [NOTES['A4'], NOTES['C4'], NOTES['E4']],  # A minor
    'Am7': [NOTES['A4'], NOTES['C4'], NOTES['E4'], NOTES['G4']],  # A minor 7th
    'F': [NOTES['F4'], NOTES['A4'], NOTES['C5']],  # F major
    'Fmaj7': [NOTES['F4'], NOTES['A4'], NOTES['C5'], NOTES['E5']],  # F major 7th
    'G': [NOTES['G4'], NOTES['B4'], NOTES['D4']]  # G major
}

In [3]:
def generate_chord(frequencies, duration=1000, harmonics=2, detune=0.5, waveform='sine'):
    """Generate a chord with harmonics and detuning for warmth."""
    wave_gen = {'sine': Sine, 'sawtooth': Sawtooth, 'square': Square}[waveform]
    chord = AudioSegment.silent(duration=0)

    for freq in frequencies:
        base = wave_gen(freq).to_audio_segment(duration=duration)
        layers = [base.fade_in(20).fade_out(20)]  # Base waveform with fades

        # Add harmonics and detuned layers
        for i in range(1, harmonics + 1):
            harmonic_freq = freq * (i + 1)
            detuned_freqs = [harmonic_freq - detune, harmonic_freq + detune]
            for dfreq in detuned_freqs:
                layers.append(wave_gen(dfreq).to_audio_segment(duration=duration).fade_in(20).fade_out(20))

        # Combine all layers for this note
        combined = sum(layers)
        chord += combined

    return chord

def apply_reverb(audio_segment, intensity=50):
    """Apply reverb effect to an AudioSegment."""
    audio_segment.export("temp_input.wav", format="wav")
    subprocess.run([
        "sox", "temp_input.wav", "temp_reverb.wav", "reverb", str(intensity)
    ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    return AudioSegment.from_file("temp_reverb.wav", format="wav")

def play_audio_segment(audio_segment):
    """Play an AudioSegment directly using ffplay via subprocess."""
    audio_segment.export("temp_chord.wav", format="wav")
    subprocess.run(["ffplay", "-nodisp", "-autoexit", "temp_chord.wav"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

def play_chord(chord_name, duration=1000, waveform='sine', harmonics=2, detune=0.5):
    """Play a single named chord with advanced sound generation."""
    if chord_name in CHORDS:
        print(f"Playing {chord_name} chord (Waveform: {waveform})...")
        chord = generate_chord(CHORDS[chord_name], duration=duration, harmonics=harmonics, detune=detune, waveform=waveform)
        chord = apply_reverb(chord, intensity=60)
        play_audio_segment(chord)
    else:
        print(f"Chord {chord_name} not found!")

def play_progression(progression, duration=1000, delay=0.3, waveform='sine', harmonics=2, detune=0.5):
    """Play a chord progression with enhanced sound generation."""
    full_audio = AudioSegment.silent(duration=0)
    for chord_name in progression:
        if chord_name in CHORDS:
            chord = generate_chord(
                CHORDS[chord_name], duration=duration,
                harmonics=harmonics, detune=detune, waveform=waveform
            )
            chord = apply_reverb(chord, intensity=60)  # Add reverb
            full_audio += chord + AudioSegment.silent(duration=int(delay * 1000))
        else:
            print(f"Chord {chord_name} not found in progression!")
    play_audio_segment(full_audio)

## Waveform Exploration

Explore how different waveforms affect the emotional quality of chords!

In [ ]:
# Waveform Comparison
print("Comparing Waveforms for C Major Chord")
for waveform in [ 'square']: # 'sine', 'sawtooth',
    play_chord('C', waveform=waveform, harmonics=1, detune=3.0)

## Extended Chord Exploration

Discover the rich textures of extended chords with different waveforms!

In [ ]:
# Extended Chord Progression
print("Rich Jazz-Inspired Progression")
jazz_progression = ['Cmaj7', 'Am7', 'Fmaj7', 'G9']
play_progression(jazz_progression, duration=100, delay=0.54, waveform='sawtooth', harmonics=3, detune=4.5)

## Create Your Own!

Experiment with chord progressions, waveforms, and sound generation parameters.

In [ ]:
# Custom Progression Template
custom = [
    'Cmaj7',   # Start with a rich major 7th
    'Am7',     # Move to a minor 7th
    'Fmaj7',   # Then a major 7th
    'G9'       # End on a 9th chord
]

# Uncomment and modify to play:
play_progression(custom, duration=300, delay=0.54, waveform='sawtooth', harmonics=1, detune=4.5)